#### 📁 Data Collection & Understanding

In [0]:
# /FileStore/tables/sales_data.csv
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
spark = SparkSession.builder.appName("SalesAnalysis")\
                            .getOrCreate()

In [0]:
df = spark.read.csv("/FileStore/tables/sales_data.csv", 
                    header=True,
                    inferSchema=True)

In [0]:
df.columns

Out[4]: ['Region',
 'Country',
 'Item Type',
 'Sales Channel',
 'Order Priority',
 'Order Date',
 'Order ID',
 'Ship Date',
 'Units Sold',
 'Unit Price',
 'Unit Cost',
 'Total Revenue',
 'Total Cost',
 'Total Profit']

In [0]:
df.dtypes

Out[5]: [('Region', 'string'),
 ('Country', 'string'),
 ('Item Type', 'string'),
 ('Sales Channel', 'string'),
 ('Order Priority', 'string'),
 ('Order Date', 'date'),
 ('Order ID', 'int'),
 ('Ship Date', 'date'),
 ('Units Sold', 'int'),
 ('Unit Price', 'double'),
 ('Unit Cost', 'double'),
 ('Total Revenue', 'double'),
 ('Total Cost', 'double'),
 ('Total Profit', 'double')]

#### 🧹 Data Cleaning & Preprocessing

In [0]:
df.na.drop() # drop null values

Out[6]: DataFrame[Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: date, Order ID: int, Ship Date: date, Units Sold: int, Unit Price: double, Unit Cost: double, Total Revenue: double, Total Cost: double, Total Profit: double]

In [0]:
df.count()

Out[7]: 1000

In [0]:
df.dropDuplicates() # drop duplicate values

Out[8]: DataFrame[Region: string, Country: string, Item Type: string, Sales Channel: string, Order Priority: string, Order Date: date, Order ID: int, Ship Date: date, Units Sold: int, Unit Price: double, Unit Cost: double, Total Revenue: double, Total Cost: double, Total Profit: double]

In [0]:
# Shipping Delay = Ship Date - Order Date
df = df.withColumn("Shipping_Delay", 
               datediff(col("Ship Date"), col("Order Date")))
df.show(3)

+-----------+-------+-----------+-------------+--------------+----------+--------+----------+----------+----------+---------+-------------+----------+------------+--------------+
|     Region|Country|  Item Type|Sales Channel|Order Priority|Order Date|Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Shipping_Delay|
+-----------+-------+-----------+-------------+--------------+----------+--------+----------+----------+----------+---------+-------------+----------+------------+--------------+
|       Asia|  China|  Beverages|       Online|             C|2023-07-25|  148411|2023-07-28|        81|    179.66|   154.54|     14552.46|  12517.74|     2034.72|             3|
|Middle East| Turkey|Electronics|      Offline|             C|2023-05-27|  189611|2023-05-31|        40|     93.93|    82.88|       3757.2|    3315.2|       442.0|             4|
|       Asia|  Japan|  Beverages|      Offline|             H|2023-03-24|  588133|2023-03-27|       357| 

In [0]:
# Profit Margin = Total Profit / Total Revenue
df = df.withColumn("Profit Margin",
              round(col("Total Profit") / col("Total Revenue"), 2))
df.show(3)

+-----------+-------+-----------+-------------+--------------+----------+--------+----------+----------+----------+---------+-------------+----------+------------+--------------+-------------+
|     Region|Country|  Item Type|Sales Channel|Order Priority|Order Date|Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|Shipping_Delay|Profit Margin|
+-----------+-------+-----------+-------------+--------------+----------+--------+----------+----------+----------+---------+-------------+----------+------------+--------------+-------------+
|       Asia|  China|  Beverages|       Online|             C|2023-07-25|  148411|2023-07-28|        81|    179.66|   154.54|     14552.46|  12517.74|     2034.72|             3|         0.14|
|Middle East| Turkey|Electronics|      Offline|             C|2023-05-27|  189611|2023-05-31|        40|     93.93|    82.88|       3757.2|    3315.2|       442.0|             4|         0.12|
|       Asia|  Japan|  Beverages|  

#### 🔝 Top profitable products by region

In [0]:
profit_by_product_region = df.groupBy("Region", "Item Type").agg(sum("Total Profit").alias("total_profit"))
profit_by_product_region.show(5)

+------------------+---------------+------------------+
|            Region|      Item Type|      total_profit|
+------------------+---------------+------------------+
|              Asia|      Beverages| 285860.8599999999|
|     South America|      Household|         150865.67|
|            Europe|Office Supplies|183455.42000000004|
|Sub-Saharan Africa|Office Supplies|         123255.78|
|       Middle East|      Household| 292048.9099999999|
+------------------+---------------+------------------+
only showing top 5 rows



In [0]:
from pyspark.sql.window import Window

window = Window.partitionBy("Region").orderBy(desc("total_profit"))
rank_product = profit_by_product_region.withColumn("rank", row_number().over(window))

# top 3 products
top_region_product = rank_product.filter(col("rank")<=3)
top_region_product.show(truncate=False)

+------------------+---------------+------------------+----+
|Region            |Item Type      |total_profit      |rank|
+------------------+---------------+------------------+----+
|Asia              |Household      |292510.16000000003|1   |
|Asia              |Beverages      |285860.8599999999 |2   |
|Asia              |Office Supplies|285775.13         |3   |
|Europe            |Electronics    |281476.89         |1   |
|Europe            |Household      |254305.72999999998|2   |
|Europe            |Clothing       |184641.58         |3   |
|Middle East       |Household      |292048.9099999999 |1   |
|Middle East       |Clothing       |255982.81000000008|2   |
|Middle East       |Office Supplies|254063.71999999994|3   |
|North America     |Electronics    |359804.66000000003|1   |
|North America     |Beverages      |302849.25         |2   |
|North America     |Household      |258328.85         |3   |
|South America     |Office Supplies|351372.1100000001 |1   |
|South America     |Elec

#### 📈 Monthly revenue and profit trends

In [0]:
df = df.withColumn("order_date",to_date(col("Order Date"), "yyyy-MM-dd")) # format date
df = df.withColumn("yearMonth", date_format(col("order_date"), "yyyy-MM")) # get year month for grouping

monthly_revenue = df.groupBy("yearMonth")\
                    .agg(
                        round(sum("Total Revenue"), 2).alias("sum_total_revenue"),
                        round(sum("Total Profit"), 2).alias("sum_total_profit")
                    )\
                    .orderBy("yearMonth")

monthly_revenue.show(truncate=False)

+---------+-----------------+----------------+
|yearMonth|sum_total_revenue|sum_total_profit|
+---------+-----------------+----------------+
|2023-01  |1976034.7        |580340.59       |
|2023-02  |1732755.41       |490489.42       |
|2023-03  |2590025.48       |840257.36       |
|2023-04  |2162066.96       |680673.49       |
|2023-05  |2409738.92       |740414.78       |
|2023-06  |1987871.03       |546489.01       |
|2023-07  |2096731.69       |585334.33       |
|2023-08  |2256981.52       |728944.82       |
|2023-09  |2288771.0        |690549.44       |
|2023-10  |1931730.31       |608902.78       |
|2023-11  |2374191.0        |698853.53       |
|2023-12  |2198262.99       |695109.28       |
|2024-01  |47058.3          |23398.65        |
+---------+-----------------+----------------+



#### 📦 Average shipping time by country

In [0]:
df = df.withColumn("order_date", to_date(col("Order Date"), "yyyy-MM-dd"))\
        .withColumn("shipping_date", to_date(col("Ship Date"), "yyy-MM-dd"))

df = df.withColumn("shipping_delay", datediff("shipping_date", "order_date"))\
        
avg_shippingTime_country = df.groupBy("Country")\
                             .agg(avg("shipping_delay").alias("average_shipping_time"))\
                             .orderBy(col("average_shipping_time").asc())

avg_shippingTime_country.show()

+--------------+---------------------+
|       Country|average_shipping_time|
+--------------+---------------------+
|        France|    7.275862068965517|
|         China|   7.7560975609756095|
|   South Korea|    7.868421052631579|
|         Kenya|               7.9375|
|         India|    7.948717948717949|
|       Nigeria|                  8.0|
|      Colombia|    8.029411764705882|
|       Germany|    8.076923076923077|
|         Italy|    8.076923076923077|
|         Spain|    8.137931034482758|
|        Brazil|     8.23076923076923|
|United Kingdom|    8.444444444444445|
|  Saudi Arabia|    8.483870967741936|
|         Chile|    8.535714285714286|
|        Mexico|    8.580645161290322|
|         Ghana|                 8.65|
|        Canada|     8.65079365079365|
|        Turkey|                 8.75|
|         Japan|    8.755555555555556|
|     Argentina|    9.105263157894736|
+--------------+---------------------+
only showing top 20 rows



#### 💰 Compare Online vs. Offline sales performance

In [0]:
online_offline_sale = df.groupBy("Sales Channel")\
                        .agg(
                            round(sum("Total Revenue"), 2).alias("sum_total_revenue"),
                            round(sum("Total Profit"), 2).alias("sum_total_profit"),
                            countDistinct("Order ID").alias("count")
                            )\
                        .withColumn("avg_order_value", round(col("sum_total_revenue")/col("count") ,2))\
                        .orderBy("Sales Channel")

# 💡 What if you want to see the revenue & as a regular number?
online_offline_sale = online_offline_sale.withColumn("sum_total_revenue", format_number(col("sum_total_revenue"), 2))\
                                        .withColumn("sum_total_profit", format_number(col("sum_total_profit"), 2))

online_offline_sale.show()

+-------------+-----------------+----------------+-----+---------------+
|Sales Channel|sum_total_revenue|sum_total_profit|count|avg_order_value|
+-------------+-----------------+----------------+-----+---------------+
|      Offline|    12,302,090.56|    3,716,455.25|  459|       26801.94|
|       Online|    13,750,128.75|    4,193,302.23|  541|       25416.13|
+-------------+-----------------+----------------+-----+---------------+



#### 🚚 Effect of Order Priority on delivery time

In [0]:
order_priority_summary = df.groupBy("Order Priority")\
                            .agg(
                                round(avg("shipping_delay"), 2).alias("avg_delivery_time"),
                                min("shipping_delay").alias("min_days"),
                                max("shipping_delay").alias("max_days")
                            )

order_priority_summary.show()

+--------------+-----------------+--------+--------+
|Order Priority|avg_delivery_time|min_days|max_days|
+--------------+-----------------+--------+--------+
|             L|             8.59|       2|      15|
|             M|              8.8|       2|      15|
|             C|             8.52|       2|      15|
|             H|             8.17|       2|      15|
+--------------+-----------------+--------+--------+



#### 📉 Low-margin high-selling products

In [0]:
product_sales = df.groupBy("Item Type")\
                  .agg(
                      sum("Units Sold").alias("total_unit_sold"),
                      round(avg("Profit Margin"), 2).alias("avg_profit_margin"),
                      round(sum("Total Profit"), 2).alias("sum_total_profit")
                  )\
                 .filter("avg_profit_margin > 0.10")\
                 .orderBy(col('avg_profit_margin').desc())\
                 .limit(3)
product_sales.show()

+---------------+---------------+-----------------+----------------+
|      Item Type|total_unit_sold|avg_profit_margin|sum_total_profit|
+---------------+---------------+-----------------+----------------+
|Office Supplies|          44458|             0.32|      1396622.55|
|      Baby Food|          38710|              0.3|      1129383.87|
|    Electronics|          44219|              0.3|      1451169.58|
+---------------+---------------+-----------------+----------------+

